# Project 1 Phylogenetics

## Outline

1. The Big Picture Introduction and Overview
2. Distance functions
    * Jukes Cantor
    * Kimura-2
    * Kimura-3
3. UPGMA algorithm
4. Neighbor Joining Algorithm
5. Mutate function for testing
6. Comparisons with Data
7. Conclusion

### "The Big Picture" Introduction and Overview

This project investigates a few of the many mathematical and computational techniques used to model evolution between different species. The algorithms compared were $\textbf{UPGMA (Unweighted Pair Group Method with Arithmetic Mean)}$ and $\textbf{Neighbor-Joining}$. Further, these phylogenetic inference algorithms yielded trees according to evolution simulated by these three different distance methods: the $\textbf{Jukes-Cantor}$, $\textbf{Kimura-2}$, and $\textbf{Kimura-3}$ Markov models (Markov models are models  with the property that changes imposed on the future state of the system is dependent on only present state and not the many previous states before). 

 To begin, evolution was simulated using a function, $\texttt{mutate (A, t, seq)}$ , on sample DNA sequences. This function "mutated" the aforementioned sequence according to time and the individual parameters given by each distance method. For instance, when using the function $\texttt{mutate()}$ to simulate evolution according to the Jukes-Cantor model, Jukes-Cantor distances between the ancestor and progeny sequences were computed and compared to the genuine distances for accuracy. Once these distances were calculated, phylogenetic inference algorithms, such as the neighbor-joining algorithm, were implemented to produce phylogenetic trees. These trees, like the distances, were then compared to the proven, legitimate tree. This was extended by simulating evolution according to the different distance methods and then implementing the two inference algorithms on both to get the best tree.

The first and simplest mathematical model used was the $\textbf{Jukes-Cantor}$ model. The Jukes-Cantor model begins by assuming all bases (nucleotides) within a DNA sequence occur with equal probability: $\frac{1}{4}$. Secondly, it also assumes that the conditional probabilities of the observable base substitutions are the same. This means the likelihood of the purine (Adenine($\textbf{A}$) and Guanine($\textbf{G}$) being substituted by a pyrimidine (Cytosine($\textbf{C}$) and Thymine($\textbf{T}$)) has equal probability. The pitfall of the second assumption is that sort of substitution is highly unlikely because of the steric hindrance and chemical properties between their molecular structures. Lastly, this model--and the other ones as well--adopts the $\textit{molecular clock}$ assumption which presumes that DNA mutation rates of observable substitutions, $\alpha$, are constant. Realistically, the rates may not be constant, since it has been shown that the rates can be dependent on whether DNA is noncoding or coding and can change based upon the time and location of a particular sequence within DNA. Despite making many assumptions, this model was useful in that it allows preliminary estimation calculations to be made. 

Unlike the Jukes-Cantor model, the $\textbf{Kimura}$ models consider more than one parameter to compute the distances between ancestor and progeny DNA sequences. In addition to mutation rates, the $\textbf{Kimura-2}$ model incorporates different rates of transitions, $\beta$, (e.g.purine $\longleftrightarrow$ purine) and different rates of transversions, $\gamma$ (e.g.purine $\longleftrightarrow$ pyrimidine). Biologically, there are two types of transversions: an exchange between one-ring and two-ring structures.The $\textbf{Kimura-3}$ takes this into account by considering a third parameter, $\delta$, for the rates of two-ring transversions. Distances between initial and final sequences were compared using both models that were used in the phylogenetic inference algorithms.

The next area of comparison is based upon the difference between two phylogenetic inference algorithms: Neighbor-Joining and UPGMA. So in total we will have 6 different models that we will compare with one another.


## Distance Functions

### *Jukes-Cantor Distance*

In [14]:
import numpy as np
from numpy import linalg as LA

In [15]:
"""
Constructs a Jukes Cantor transition Matrix with a specified alpha level a
Args:
     a: alpha level for the Jukes Cantor Matrix
Returns:
     Transition Matrix corresponding to the Jukes-Cantor Algorithm
"""

def JC_matrix(a):

    """
    >>> np.trace(JC_matrix(.25))
    3.0
    """

    b = a/3
    M = np.array([[1-a, b, b, b],
                 [b, 1-a, b, b],
                 [b, b, 1-a, b],
                 [b, b, b, 1-a]])
    return M


"""
Computes proportion of differing letters from two strings of the same size
Args:
    s1: string 1
    s2: string 2 
Returns:
    Throws error if the strings are not of the same length
    Else, returns proportion (in between 0 and 1) of differing letters
"""

def prop_diff(s1,s2):
    # """ LOOK curious as to why this isn't working
    # >>> prop_diff("ATTGAC","ATGGCC") 
    # float(2)/float(6)  
    # """
    if len(s1) != len(s2):
        raise ValueError("Cannot compute compare DNA sequences of differing lenth")
    diffs = 0
    i     = 0
    while i < len(s1):
        if s1[i] != s2[i]:
            diffs += 1
        i += 1
    return float(diffs)/float(len(s1))

"""
Computes the JC distance between two sequences.
Args:
    s1: string 1
    s2: string 2 
Returns:
    Throws error if the strings are not of the same length
    Else, computes JC distance
"""

def JC_distance(s1,s2):
    prop_diff = prop_diff(s1,s2)
    return 1 - (np.log(1 - 4/3*prop_diff))


## UPGMA Algorithm

In [16]:
def closest_neighbors(M):
    size = len(M)
    min = 100000
    coordinates = (0,0)
    for i in xrange(size-1):
        for j in xrange(i + 1, size):
            if M[i,j] <= min:
                min = M[i,j]
                coordinates = (i,j)
    return coordinates

"""
Computes the subsequent Distance Matrix of the UPGMA algorithm
Args:
    M: the transition Matrix for the Jukes Cantor Algorithm
Returns:
    subsequent matrix using UPGMA
"""
def update_UPGMA_matrix(M, taxa1, taxa2):
        size  = len(M)
        new_size = size - 1
        ans       = np.zeros((new_size, new_size))

        # will use the 0th row and column for the new species in the matrix
        #
        # copies over the vales from the old matrix
        computed_col = 1
        new_row = 1
        for i in xrange(size - 1):
            if i != taxa1 and i != taxa2:
                new_col = new_row + 1
                for j in xrange(i + 1, size):
                   if j != taxa1 and j != taxa2:
                       ans[new_row,new_col] = M[i,j]
                       new_col += 1
                new_row +=1
        # compute the first row of entries
        new_col = 1
        for j in xrange(size):
            if j != taxa1 and j != taxa2:
                #exploits the fact that we have an upper triangular so col > row always
                ans[0,new_col] = (M[min(taxa1, j), max(taxa1, j)] + M[min(taxa2, j), max(taxa2, j)]) / 2
                new_col += 1
        return ans

def UPGMA(M, names):
    """
    >>> UPGMA(np.array([[0, .45, .27, .53],[0,   0, .40, .50],[0,   0,   0, .62],[0,   0,   0,  0]]))
    np.array([[0, .425, .575],[0,    0,  .50],[0,    0,    0]])
    """
    def search_nodes(trees ,name):
        for tree in trees:
            if tree.name == name:
                return tree
    trees = []
    while True:
        taxa1, taxa2 = closest_neighbors(M)
        if taxa1 > taxa2:
            tmp = taxa1
            taxa1 = taxa2
            taxa1 = tmp
        #define a new parent for the join
        t = Tree()
        #search for the children in trees and add them
        A = search_nodes(trees, names[taxa1])
        if A == None:
            A = t.add_child(name = names[taxa1])
        else:
            t.add_child(A)
            trees.remove(A)
        B = search_nodes(trees, names[taxa2])
        if B == None:
            B = t.add_child(name = names[taxa2])
        else:
            t.add_child(B)
            trees.remove(B)
        #delete old taxa names and update the new name
        new_names = [names[taxa1] + names[taxa2]]
        del names[taxa2]
        del names[taxa1]
        [new_names.append(name) for name in names]
        names = new_names
        #create the distance between children and parent
        avg_dist = M[taxa1, taxa2]/2
        A.dist = avg_dist
        B.dist = avg_dist
        #name the parent
        t.name = names[0]
        #add the new subtree
        trees.append(t)

        if len(M) <= 2:
            break
        M = update_UPGMA_matrix(M, taxa1, taxa2)
    return trees[0]

## Neighbor Joining Algorithm

## Mutate Function

In [17]:
"""
    Given a character 'A' 'G' 'C' or 'T' returns number corresponding
    to the index in whcih we need to index in our vector
    Args:
        nuc: character 'A' 'G' 'C' or 'T'
    Returns:
        number 0,1,2,3 indicatin what index to set to 1
"""

def DNA_to_position(nuc):
    if nuc == 'A':
        return 0
    if nuc == 'G':
        return 1
    if nuc == 'C':
        return 2
    if nuc == 'T':
        return 3

"""
    Given a number 0,1,2,3 returns character 'A' 'G' 'C' or 'T'
    corresponding to the nucleic acid represented in the model
    Args:
        number 0,1,2,3
    Returns:
        character 'A' 'G' 'C' or 'T' orresponding to the
        nucleic acid represented in the model
"""

def position_to_DNA(pos):
    if pos == 0:
        return 'A'
    if pos == 1:
        return 'G'
    if pos == 2:
        return 'C'
    if pos == 3:
        return 'T'

"""
    Mutates the DNA string seq, int t times according to the
    Jukes-Cantor model specified by the alpha value, a
    Args:
        a:   alpha level for the Jukes Cantor Matrix
        t:   number of time steps we plan to simulate
        seq: Sequence of DNA represented as a string
    Returns:
        simulated descendant sequence after t time steps
        consitently represented as a string
"""

def mut(a, t, seq):
    M = JC_matrix(a)
    M = LA.matrix_power(M, t)
    tokens = list(seq)
    for i in xrange(len(tokens)):
        p                       = np.zeros(4)
        p[DNA_to_position(tokens[0])] = 1
        p                       = np.dot(M,p)

        rand                    = np.random.rand()
        if rand < p[0]:
            val = 'A'
        elif rand < p[0]+p[1]:
            val = 'G'
        elif rand < p[0]+p[1]+p[2]:
            val = 'C'
        else:
            val = 'T'
        tokens[i] = val
    return ''.join(tokens)

### Verification of mutate

In [22]:
for i in xrange(10):
    print(mut(.3,4,'GATTACA'))

GGTGGAG
GTGTAGG
CCGGCTC
TACTTCA
GAGGTCG
CCGTGCC
ATATCTC
CCTAGCC
CATCAGT
CTGTAAG


Above we clearly see the mutate function working on our test string GATTACA

## Comparisons with Data

## Conclusion